<a href="https://colab.research.google.com/github/qwe030609/python-web-crawler/blob/master/Assignment%2B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [0]:
import pandas as pd
from datetime import datetime
import re

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)

04/20/2009; 04/20/09; 4/20/09; 4/3/09
Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
Feb 2009; Sep 2009; Oct 2010
6/2008; 12/2009
2009; 2010

In [148]:
#  df1[2][:].str.len() == 2
# df1[2][100].replace('92', '1')
# len(df1[2][0])
df11 = pd.DataFrame({'A': [0, 0, 2, 3, 4],
                   'B': [5, 6, 0, 0, 9],
                   'C': ['a', 'a', 'c', 'd', 'e']})
df11.replace('a', 5)

,A,B,C
0,0,5,5
1,0,6,5
2,2,0,c
3,3,0,d
4,4,9,e


In [0]:
# find all dates in the data
df1 = df.str.findall(r'(\d{1,3})[/-](\d{1,2})[/-](\d{2,4})')[:125]    # 0 ~ 124
# print(df1[:50])   # 72 got 2 extracted
# print(df1[50:100])
# print(df1[100:])

df2 = df.str.findall(r'(\d{2} )?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z.]*[-, ]*(\d{2})?,* (\d{4})')[125:343]
# print(df2[:50])   # 125 ~ 342
# print(df2[50:100])
# print(df2[100:150])
# print(df2[150:200])
# print(df2[200:])

df3 = df.str.findall(r'(\d{1,3})?/*(\d{1,2})?/*([1-2]\d{3})')[343:]   # 343 ~ 500
# print(df3[:50])   # 343 ~ 500
# print(df3[50:100])
# print(df3[100:150])
# print(df3[150:])

In [224]:
from datetime import datetime
import re

# after finding all data for dates, extract to a new date dataframe
df1 = df.str.extract(r'(\d{1,3})[/-](\d{1,2})[/-](\d{2,4})')[:125]
df2 = df.str.extract(r'(\d{2} )?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z.]*[-, ]*(\d{2})?,* (\d{4})')[125:343]
df3 = df.str.extract(r'(\d{1,3})?/*(\d{1,2})?/*([1-2]\d{3})')[343:]

# print(df2.iloc[60:120])
# # df1.iloc[72:100]

# clean the data and convert to uniform form
# df1.head()
# df2.head()
# df3.head(60)

# replace year xx in 2 digits to 19xx or 20xx for 4 digits
for i in range(df1.shape[0]):
  if len(df1[2][i]) == 2 and int(df1[2][i]) > 50: 
    df1[2][i] = df1[2][i].replace(df1[2][i], str(19) + df1[2][i])
  elif len(df1[2][i]) == 2 and int(df1[2][i]) <= 50:
    df1[2][i] = df1[2][i].replace(df1[2][i], str(20) + df1[2][i])

# reorder df2
df2 = df2[[1, 0, 3]] 
df2.columns = range(df2.shape[1])

# fill missing values(NaN) with 1 
df2.fillna(value = 1, inplace = True)
df3.fillna(value = 1, inplace = True)

# replace literal month values to numbers
month_list = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
for i, month in enumerate(month_list):
  df2.replace(month, i+1, inplace = True)

# concatenate all df to a single df
df_dates = pd.concat([df1, df2, df3])

# rename columns titles to month, day, year
df_dates.columns = ['month', 'day', 'year']

# check dtype is correct (not object)
# df_dates.info()

# convert data type from str to int
df_dates = df_dates.astype('int')
# df_dates.info()
                         
# Sort the list in ascending order of dates
df_dates = df_dates['month'].map(str) + '/' + df_dates['day'].map(str) + '/' + df_dates['year'].map(str)
# df_dates.to_list().sort(key = lambda date: datetime.strptime(date, '%m/%d/%Y')) 

df_dates = pd.to_datetime(df_dates)
df_dates_sorted = df_dates.sort_values()
df_dates_sorted = df_dates_sorted.to_frame().reset_index()

# output final dates index
df_dates_sorted_index = df_dates_sorted['index']
df_dates_sorted_index.dtype


dtype('int64')

In [225]:
def date_sorter(df):

  # after finding all data for dates, extract to a new date dataframe
  df1 = df.str.extract(r'(\d{1,3})[/-](\d{1,2})[/-](\d{2,4})')[:125]
  df2 = df.str.extract(r'(\d{2} )?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z.]*[-, ]*(\d{2})?,* (\d{4})')[125:343]
  df3 = df.str.extract(r'(\d{1,3})?/*(\d{1,2})?/*([1-2]\d{3})')[343:]

  # print(df2.iloc[60:120])
  # # df1.iloc[72:100]

  # clean the data and convert to uniform form
  # df1.head()
  # df2.head()
  # df3.head(60)

  # replace year xx in 2 digits to 19xx or 20xx for 4 digits
  for i in range(df1.shape[0]):
    if len(df1[2][i]) == 2 and int(df1[2][i]) > 50: 
      df1[2][i] = df1[2][i].replace(df1[2][i], str(19) + df1[2][i])
    elif len(df1[2][i]) == 2 and int(df1[2][i]) <= 50:
      df1[2][i] = df1[2][i].replace(df1[2][i], str(20) + df1[2][i])

  # reorder df2
  df2 = df2[[1, 0, 3]] 
  df2.columns = range(df2.shape[1])

  # fill missing values(NaN) with 1 
  df2.fillna(value = 1, inplace = True)
  df3.fillna(value = 1, inplace = True)

  # replace literal month values to numbers
  month_list = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
  for i, month in enumerate(month_list):
    df2.replace(month, i+1, inplace = True)

  # concatenate all df to a single df
  df_dates = pd.concat([df1, df2, df3])

  # rename columns titles to month, day, year
  df_dates.columns = ['month', 'day', 'year']

  # check dtype is correct (not object)
  # df_dates.info()

  # convert data type from str to int
  df_dates = df_dates.astype('int')
  # df_dates.info()
                          
  # Sort the list in ascending order of dates
  df_dates = df_dates['month'].map(str) + '/' + df_dates['day'].map(str) + '/' + df_dates['year'].map(str)
  # df_dates.to_list().sort(key = lambda date: datetime.strptime(date, '%m/%d/%Y')) 

  df_dates = pd.to_datetime(df_dates)
  df_dates_sorted = df_dates.sort_values()
  df_dates_sorted = df_dates_sorted.to_frame().reset_index()

  # output final dates index
  df_dates_sorted_index = df_dates_sorted['index']

  return df_dates_sorted_index
date_sorter(df)  

ERROR! Session/line number was not unique in database. History logging moved to new session 69


0        9
1       84
2        2
3       53
4       28
      ... 
495    231
496    141
497    186
498    161
499    413
Name: index, Length: 500, dtype: int64